# Vision-Caption Projector Training

This notebook trains the projector on COCO captions.

Setup checklist:
- Enable GPU
- Enable Internet (Kaggle)

Run cells in order.

In [ ]:
# Step 1: Clone and install
import os
import subprocess
import sys

if not os.path.isdir("vision-caption"):
    subprocess.run(
        ["git", "clone", "https://github.com/asynced24/vision-caption.git"],
        check=True,
    )

os.chdir("vision-caption")
subprocess.run([sys.executable, "-m", "pip", "install", "-e", ".", "-q"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "pycocotools", "-q"], check=True)
print("Setup complete")

In [ ]:
# Step 2: Download COCO dataset (with progress bar)
from pathlib import Path
import subprocess


data_dir = Path("coco_data")
data_dir.mkdir(exist_ok=True)

train_zip = data_dir / "train2017.zip"
ann_zip = data_dir / "annotations_trainval2017.zip"

if not (data_dir / "train2017").exists():
    print("Downloading train2017 images...")
    subprocess.run(
        [
            "wget",
            "--progress=bar:force:noscroll",
            "-O",
            str(train_zip),
            "http://images.cocodataset.org/zips/train2017.zip",
        ],
        check=True,
    )
    subprocess.run(["unzip", "-q", str(train_zip), "-d", str(data_dir)], check=True)
    train_zip.unlink(missing_ok=True)

if not (data_dir / "annotations").exists():
    print("Downloading annotations...")
    subprocess.run(
        [
            "wget",
            "--progress=bar:force:noscroll",
            "-O",
            str(ann_zip),
            "http://images.cocodataset.org/annotations/annotations_trainval2017.zip",
        ],
        check=True,
    )
    subprocess.run(["unzip", "-q", str(ann_zip), "-d", str(data_dir)], check=True)
    ann_zip.unlink(missing_ok=True)

print("Dataset ready")

In [ ]:
# Step 3: Train projector
import subprocess
import sys

subprocess.run(
    [
        sys.executable,
        "train.py",
        "--images-dir",
        "coco_data/train2017",
        "--annotations-file",
        "coco_data/annotations/captions_train2017.json",
        "--output-dir",
        "checkpoints",
        "--epochs",
        "3",
        "--batch-size",
        "32",
        "--lr",
        "1e-3",
    ],
    check=True,
)
print("Training complete")

In [ ]:
# Step 4: Quick test
from vision_caption import ModelConfig, load_model
from PIL import Image
import requests
from io import BytesIO

config = ModelConfig()
config.projector_path = "checkpoints/projector_final.pt"
model = load_model(config)

url = "https://images.unsplash.com/photo-1518791841217-8f162f1e1131"
image = Image.open(BytesIO(requests.get(url).content))

print("Caption:", model.generate(image))
display(image)

In [ ]:
# Step 5: Download weights
try:
    from google.colab import files

    files.download("checkpoints/projector_final.pt")
    print("Downloaded from Colab")
except Exception:
    print("Kaggle: download from the Output tab (checkpoints/projector_final.pt)")

# Vision-Caption Projector Training

Train the projector on COCO Captions. Works on **Colab** and **Kaggle**.

**IMPORTANT:** Enable GPU + Internet first!
- **Colab:** Runtime → Change runtime type → T4 GPU
- **Kaggle:** Settings → Accelerator → GPU T4 x2 + Internet ON

**If Kaggle logs show `Accelerator: None`, GPU is not enabled.**
**If `git clone` fails with `Could not resolve host`, Internet is OFF.**

In [ ]:
# Step 1: Clone and install
!git clone https://github.com/asynced24/vision-caption.git
%cd vision-caption
%pip install -e . -q
%pip install pycocotools -q
print("✓ Setup complete")

In [ ]:
# Step 2: Download COCO dataset (~19GB)
from pathlib import Path

data_dir = Path("coco_data")
data_dir.mkdir(exist_ok=True)

if not (data_dir / "train2017").exists():
    print("Downloading images...")
    !wget -q http://images.cocodataset.org/zips/train2017.zip -P coco_data
    !unzip -q coco_data/train2017.zip -d coco_data
    !rm coco_data/train2017.zip

if not (data_dir / "annotations").exists():
    print("Downloading annotations...")
    !wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P coco_data
    !unzip -q coco_data/annotations_trainval2017.zip -d coco_data
    !rm coco_data/annotations_trainval2017.zip

print("✓ Dataset ready!")

In [ ]:
# Step 3: Train projector (full COCO)
!python train.py --images-dir coco_data/train2017 --annotations-file coco_data/annotations/captions_train2017.json --output-dir checkpoints --epochs 3 --batch-size 32 --lr 1e-3

In [ ]:
# Step 4: Test trained model
from vision_caption import ModelConfig, load_model
from PIL import Image
import requests
from io import BytesIO

config = ModelConfig()
config.projector_path = "checkpoints/projector_final.pt"
model = load_model(config)

# Test image
url = "https://images.unsplash.com/photo-1518791841217-8f162f1e1131"
image = Image.open(BytesIO(requests.get(url).content))

print(f"Caption: {model.generate(image)}")
display(image)

In [ ]:
# Step 5: Download trained weights
try:
    from google.colab import files
    files.download('checkpoints/projector_final.pt')
    print("✓ Downloaded (Colab)")
except ImportError:
    print("✓ On Kaggle: Click Output tab → Download projector_final.pt")